# Tablas de Cambio de Opinión - Formato Excel

Este notebook genera las mismas tablas que el archivo 45, pero exportadas a Excel con formato completo (colores, estilos, etc.).

**Ventajas del formato Excel:**
- Mejor calidad visual
- Editable si es necesario
- Celdas realmente combinadas
- Formato profesional
- Fácil de compartir e incluir en documentos

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import kruskal
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

## 1. Cargar Datos y Calcular Estadísticas

Igual que en el archivo 45, cargamos los datos desde Excel y calculamos los diccionarios necesarios.

In [ ]:
# Mapa de colores para las categorías/poblaciones (del archivo 33)
Mapa_Colores_Categorias = {
    'Left_Wing': '#f65058',              # Rojo
    'Progressivism': '#0078bf',          # Azul
    'Centre': '#009cdd',                 # Celeste
    'Moderate_Right_A': '#f7d117',       # Amarillo
    'Moderate_Right_B': '#f7d117',       # Amarillo
    'Right_Wing_Libertarian': '#753bbd'  # Morado
}

In [ ]:
# Rutas a los archivos Excel
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames desde Excel
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados desde Excel:")
print(f"  - Generales: {len(df_Generales)} registros, {len(df_Generales.columns)} columnas")
print(f"  - Ballotage: {len(df_Ballotage)} registros, {len(df_Ballotage.columns)} columnas")

In [ ]:
def Calcular_Diccionario_Resultados_CO(dfs_Finales):
    """
    Calcula estadísticas descriptivas para cada categoría y variable de CO.
    """
    
    # Definir ítems
    Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
    Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
    
    # Generar nombres de columnas
    Columnas_CO_Pro = [f'CO_Item_{Item}_Izq' for Item in Items_Progresistas] + \
                      [f'CO_Item_{Item}_Der' for Item in Items_Progresistas]
    
    Columnas_CO_Con = [f'CO_Item_{Item}_Izq' for Item in Items_Conservadores] + \
                      [f'CO_Item_{Item}_Der' for Item in Items_Conservadores]
    
    Columnas_Cambios = Columnas_CO_Pro + Columnas_CO_Con
    
    # Inicializar diccionario de resultados
    Diccionario_Resultados = {'Generales': {}, 'Ballotage': {}}
    
    # Categorías válidas
    Categorias_Validas = [
        'Left_Wing', 
        'Progressivism', 
        'Centre',
        'Moderate_Right_A', 
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]
    
    # Para cada dataset
    for Nombre_df, df in dfs_Finales.items():
        # Filtrar solo las categorías válidas
        df_Descriptivo = df[df['Categoria_PASO_2023'].isin(Categorias_Validas)]
        
        # Para cada categoría
        for Categoria in Categorias_Validas:
            Datos_Categoria = df_Descriptivo[df_Descriptivo['Categoria_PASO_2023'] == Categoria]
            Estadisticas_Categoria = {}
            
            # Para cada columna de cambio de opinión
            for Columna in Columnas_Cambios:
                if Columna in df.columns:  # Verificar que la columna existe
                    Valores = Datos_Categoria[Columna].dropna()
                    
                    # Calcular estadísticas
                    if len(Valores) > 0:
                        Media = Valores.mean()
                        Mediana = Valores.median()
                        Desvio_Estandar = Valores.std()
                        Error_Estandar = Valores.sem()
                        N = Valores.count()
                    else:
                        Media = 0
                        Mediana = 0
                        Desvio_Estandar = 0
                        Error_Estandar = 0
                        N = 0
                    
                    Estadisticas_Categoria[Columna] = {
                        'Media': Media,
                        'Mediana': Mediana,
                        'Desvio_Estandar': Desvio_Estandar,
                        'Error_Estandar': Error_Estandar,
                        'N': N
                    }
            
            Diccionario_Resultados[Nombre_df][Categoria] = Estadisticas_Categoria
    
    return Diccionario_Resultados


def Calcular_Diccionario_P_Valores(dfs_Finales):
    """
    Calcula p-valores usando el test de Kruskal-Wallis.
    """
    
    # Definir ítems
    Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
    Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
    
    # Generar nombres de columnas
    Columnas_CO_Pro = [f'CO_Item_{Item}_Izq' for Item in Items_Progresistas] + \
                      [f'CO_Item_{Item}_Der' for Item in Items_Progresistas]
    
    Columnas_CO_Con = [f'CO_Item_{Item}_Izq' for Item in Items_Conservadores] + \
                      [f'CO_Item_{Item}_Der' for Item in Items_Conservadores]
    
    Columnas_Cambios = Columnas_CO_Pro + Columnas_CO_Con
    
    # Inicializar diccionario de p-valores
    Diccionario_P_Valores = {'Generales': {}, 'Ballotage': {}}
    
    # Categorías válidas
    Categorias_Validas = [
        'Left_Wing', 
        'Progressivism', 
        'Centre',
        'Moderate_Right_A', 
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]
    
    # Para cada dataset
    for Nombre_df, df in dfs_Finales.items():
        # Filtrar solo las categorías válidas
        df_Filtrado = df[df['Categoria_PASO_2023'].isin(Categorias_Validas)]
        
        # Para cada columna de cambio de opinión
        for Nombre_Columna in Columnas_Cambios:
            if Nombre_Columna in df.columns:  # Verificar que la columna existe
                # Crear grupos por categoría
                Grupos = [
                    df_Filtrado[df_Filtrado['Categoria_PASO_2023'] == Categoria][Nombre_Columna].dropna()
                    for Categoria in Categorias_Validas
                ]
                
                # Verificar que todos los grupos tengan al menos dos datos
                if all(len(Grupo) > 1 for Grupo in Grupos):
                    try:
                        Estadistico, P_Valor = kruskal(*Grupos)
                        Diccionario_P_Valores[Nombre_df][Nombre_Columna] = P_Valor
                    except:
                        # Si falla el test, asignar NaN
                        Diccionario_P_Valores[Nombre_df][Nombre_Columna] = np.nan
                else:
                    # Si no hay suficientes datos, asignar NaN
                    Diccionario_P_Valores[Nombre_df][Nombre_Columna] = np.nan
    
    return Diccionario_P_Valores


# Calcular diccionarios
print("\n📊 Calculando estadísticas descriptivas...")
Diccionario_Resultados_CO_Individuales = Calcular_Diccionario_Resultados_CO(dfs_Finales)
print(f"✓ Diccionario de resultados calculado")

print("\n📊 Calculando p-valores (test de Kruskal-Wallis)...")
Diccionario_P_Valores = Calcular_Diccionario_P_Valores(dfs_Finales)
print(f"✓ Diccionario de p-valores calculado")

print("\n✅ Datos listos para generar tablas en Excel")

## 2. Funciones Auxiliares para Formato Excel

In [ ]:
def hex_to_rgb(hex_color):
    """Convierte color hexadecimal a formato RGB para openpyxl."""
    hex_color = hex_color.lstrip('#')
    return hex_color.upper()


def aplicar_borde_completo(cell):
    """Aplica bordes a todas las celdas."""
    thin_border = Border(
        left=Side(style='thin', color='000000'),
        right=Side(style='thin', color='000000'),
        top=Side(style='thin', color='000000'),
        bottom=Side(style='thin', color='000000')
    )
    cell.border = thin_border


def obtener_color_significancia(texto_sig):
    """Retorna el color de fondo según nivel de significancia."""
    if texto_sig == '***':
        return '90EE90'  # Verde oscuro
    elif texto_sig == '**':
        return 'B8F4B8'  # Verde medio
    elif texto_sig == '*':
        return 'D4F4D4'  # Verde claro
    elif texto_sig == 'ns':
        return 'FFE4E1'  # Rosa claro
    else:
        return 'FFFFFF'  # Blanco


def obtener_color_texto_significancia(texto_sig):
    """Retorna el color de texto según nivel de significancia."""
    if texto_sig == '***':
        return '006400'  # Verde oscuro
    elif texto_sig == '**':
        return '228B22'  # Verde medio
    elif texto_sig == '*':
        return '000000'  # Negro
    elif texto_sig == 'ns':
        return '8B0000'  # Rojo oscuro
    else:
        return '000000'  # Negro

## 3. Función 1: Tabla por Ítem

Genera una tabla para UN ítem específico mostrando TODAS las categorías comparando Generales vs Ballotage.

In [ ]:
def Crear_Tabla_Excel_Por_Item(
    Diccionario_Resultados,
    Diccionario_P_Valores,
    Numero_Item,
    Nombre_Archivo = None,
    Carpeta_Destino = 'Tablas_Excel'
):
    """
    Crea una tabla en Excel para UN ítem específico mostrando TODAS las categorías.
    Compara Generales vs Ballotage.
    """
    
    if not os.path.exists(Carpeta_Destino):
        os.makedirs(Carpeta_Destino)
    
    if Nombre_Archivo is None:
        Nombre_Archivo = f'Tabla_Item_{Numero_Item}_Todas_Categorias.xlsx'
    
    # Crear workbook
    wb = Workbook()
    ws = wb.active
    ws.title = f'Item {Numero_Item}'
    
    # Categorías
    Categorias = [
        'Left_Wing',
        'Progressivism',
        'Centre',
        'Moderate_Right_A',
        'Moderate_Right_B',
        'Right_Wing_Libertarian'
    ]
    
    Etiquetas_Categorias = {
        'Left_Wing': 'Left Wing',
        'Progressivism': 'Progressivism',
        'Centre': 'Centre',
        'Moderate_Right_A': 'Moderate Right A',
        'Moderate_Right_B': 'Moderate Right B',
        'Right_Wing_Libertarian': 'Right Wing Libertarian'
    }
    
    # Variables
    Var_Izq = f'CO_Item_{Numero_Item}_Izq'
    Var_Der = f'CO_Item_{Numero_Item}_Der'
    
    # Fila actual
    fila = 1
    
    # ENCABEZADO PRINCIPAL - Generales y Ballotage
    ws.merge_cells(start_row=fila, start_column=1, end_row=fila, end_column=1)
    ws.cell(fila, 1).value = ''
    ws.cell(fila, 1).fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    
    ws.merge_cells(start_row=fila, start_column=2, end_row=fila, end_column=4)
    cell_gen = ws.cell(fila, 2)
    cell_gen.value = 'Generales'
    cell_gen.font = Font(bold=True, color='FFFFFF', size=12)
    cell_gen.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    cell_gen.alignment = Alignment(horizontal='center', vertical='center')
    
    ws.merge_cells(start_row=fila, start_column=5, end_row=fila, end_column=7)
    cell_bal = ws.cell(fila, 5)
    cell_bal.value = 'Ballotage'
    cell_bal.font = Font(bold=True, color='FFFFFF', size=12)
    cell_bal.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
    cell_bal.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 1
    
    # SUB-ENCABEZADO
    headers = ['Categoría', 'Media±DE', 'n', 'Sig', 'Media±DE', 'n', 'Sig']
    for col_idx, header in enumerate(headers, start=1):
        cell = ws.cell(fila, col_idx)
        cell.value = header
        cell.font = Font(bold=True, color='FFFFFF', size=10)
        cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')
    
    fila += 1
    
    # DATOS - Para cada categoría
    for Categoria in Categorias:
        Etiqueta = Etiquetas_Categorias.get(Categoria, Categoria)
        Color_Cat = hex_to_rgb(Mapa_Colores_Categorias.get(Categoria, '#E7E6E6'))
        
        # Fila Izquierda
        ws.cell(fila, 1).value = f'{Etiqueta} (Izq)'
        ws.cell(fila, 1).font = Font(bold=True, color='FFFFFF')
        ws.cell(fila, 1).fill = PatternFill(start_color=Color_Cat, end_color=Color_Cat, fill_type='solid')
        ws.cell(fila, 1).alignment = Alignment(horizontal='center', vertical='center')
        
        col = 2
        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None
            
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Izq in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Izq]
            
            if Dataset in Diccionario_P_Valores:
                if Var_Izq in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Izq]
            
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)
                
                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = str(int(N))
                    
                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'
            
            # Media±DE
            ws.cell(fila, col).value = Texto_Media
            ws.cell(fila, col).alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # n
            ws.cell(fila, col).value = Texto_N
            ws.cell(fila, col).alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # Significancia
            cell_sig = ws.cell(fila, col)
            cell_sig.value = Texto_Sig
            cell_sig.alignment = Alignment(horizontal='center', vertical='center')
            color_fondo = obtener_color_significancia(Texto_Sig)
            color_texto = obtener_color_texto_significancia(Texto_Sig)
            cell_sig.fill = PatternFill(start_color=color_fondo, end_color=color_fondo, fill_type='solid')
            cell_sig.font = Font(bold=True, color=color_texto)
            col += 1
        
        fila += 1
        
        # Fila Derecha
        ws.cell(fila, 1).value = f'{Etiqueta} (Der)'
        ws.cell(fila, 1).font = Font(bold=True, color='FFFFFF')
        ws.cell(fila, 1).fill = PatternFill(start_color=Color_Cat, end_color=Color_Cat, fill_type='solid')
        ws.cell(fila, 1).alignment = Alignment(horizontal='center', vertical='center')
        
        col = 2
        for Dataset in ['Generales', 'Ballotage']:
            Stats = None
            P_Valor = None
            
            if Dataset in Diccionario_Resultados:
                if Categoria in Diccionario_Resultados[Dataset]:
                    if Var_Der in Diccionario_Resultados[Dataset][Categoria]:
                        Stats = Diccionario_Resultados[Dataset][Categoria][Var_Der]
            
            if Dataset in Diccionario_P_Valores:
                if Var_Der in Diccionario_P_Valores[Dataset]:
                    P_Valor = Diccionario_P_Valores[Dataset][Var_Der]
            
            if Stats:
                Media = Stats.get('Media', np.nan)
                DE = Stats.get('Desvio_Estandar', np.nan)
                N = Stats.get('N', 0)
                
                if not np.isnan(Media):
                    Texto_Media = f'{Media:.2f}±{DE:.2f}'
                    Texto_N = str(int(N))
                    
                    if P_Valor is not None and not np.isnan(P_Valor):
                        if P_Valor < 0.001:
                            Texto_Sig = '***'
                        elif P_Valor < 0.01:
                            Texto_Sig = '**'
                        elif P_Valor < 0.05:
                            Texto_Sig = '*'
                        else:
                            Texto_Sig = 'ns'
                    else:
                        Texto_Sig = '—'
                else:
                    Texto_Media = '—'
                    Texto_N = '—'
                    Texto_Sig = '—'
            else:
                Texto_Media = '—'
                Texto_N = '—'
                Texto_Sig = '—'
            
            # Media±DE
            ws.cell(fila, col).value = Texto_Media
            ws.cell(fila, col).alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # n
            ws.cell(fila, col).value = Texto_N
            ws.cell(fila, col).alignment = Alignment(horizontal='center', vertical='center')
            col += 1
            
            # Significancia
            cell_sig = ws.cell(fila, col)
            cell_sig.value = Texto_Sig
            cell_sig.alignment = Alignment(horizontal='center', vertical='center')
            color_fondo = obtener_color_significancia(Texto_Sig)
            color_texto = obtener_color_texto_significancia(Texto_Sig)
            cell_sig.fill = PatternFill(start_color=color_fondo, end_color=color_fondo, fill_type='solid')
            cell_sig.font = Font(bold=True, color=color_texto)
            col += 1
        
        fila += 1
        fila += 1  # Fila vacía entre categorías
    
    # Aplicar bordes a todas las celdas
    for row in ws.iter_rows(min_row=1, max_row=fila-1, min_col=1, max_col=7):
        for cell in row:
            aplicar_borde_completo(cell)
    
    # Ajustar anchos de columna
    ws.column_dimensions['A'].width = 25
    ws.column_dimensions['B'].width = 18
    ws.column_dimensions['C'].width = 8
    ws.column_dimensions['D'].width = 8
    ws.column_dimensions['E'].width = 18
    ws.column_dimensions['F'].width = 8
    ws.column_dimensions['G'].width = 8
    
    # Guardar
    Ruta_Completa = os.path.join(Carpeta_Destino, Nombre_Archivo)
    wb.save(Ruta_Completa)
    
    print(f"✅ Tabla Excel del Ítem {Numero_Item} guardada en: {Ruta_Completa}")
    
    return Ruta_Completa

## 4. Ejemplo de Uso

Genera una tabla de ejemplo para el Ítem 5:

In [ ]:
# Ejemplo: Crear tabla Excel para el Ítem 5
Ruta_Tabla = Crear_Tabla_Excel_Por_Item(
    Diccionario_Resultados = Diccionario_Resultados_CO_Individuales,
    Diccionario_P_Valores = Diccionario_P_Valores,
    Numero_Item = 5
)

print(f"\n📊 Tabla de ejemplo creada exitosamente")
print(f"   Abre el archivo para ver el formato con colores")